# Evolving Election Uncertainty Pt. II 

In [ ]:
#@title
#hide
import pandas as pd 
import numpy as np
import altair as alt
import datetime as dt
alt.themes.enable('fivethirtyeight')
pd.options.mode.chained_assignment = None

In [ ]:
#@title
#hide_input
urls = ["https://raw.githubusercontent.com/srcnor/minding_constraints/master/data/Who_will_win_the_2020_U.S._presidential_election_.csv",
        "https://raw.githubusercontent.com/srcnor/minding_constraints/master/data/Who_will_win_the_2020_U.S._presidential_election__(1).csv",
        "https://raw.githubusercontent.com/srcnor/minding_constraints/master/data/Who_will_win_the_2020_U.S._presidential_election__(2).csv",
]

In [ ]:
#@title
#hide_input
csv_list = list()

for i in urls:
  csv_list.append(pd.read_csv(i, error_bad_lines=False))

df = pd.concat(csv_list)

In [ ]:
#@title
#hide_input
df['Candidate']= df['ContractName'].astype(str)
df = df[df['Candidate'].isin(['Biden', 'Trump'])]
df['Win Probability']= df['CloseSharePrice'].str.lstrip('\$').astype(float)
df['Date']= pd.to_datetime(df['Date'])
df = df[df['Date'] > pd.to_datetime('2020-11-03')]

In our [last post](http://mindingconstraints.com/2020/10/30/election-uncertainty.html), we looked at how the implied presidential election probabilities on PredictIt showed a distinct persistence throughout the presidential campaign, despite the fact that uncertainty about a future event should generally decline as the event itself approaches. 

On election night itself, PredictIt shook off that persistence.

Well actually on election night *itself* PredictIt mostly buckled under the weight of all of the traders attempting to get in on the action: 

In [42]:
#@title
#hide_input
df_vol = df.groupby("Date", as_index = False).sum("TradeVolume")

alt.Chart(df_vol).mark_bar(color='rgb(74,82,90').encode(
    x=alt.X('Date', axis=alt.Axis(title='')),
    y=alt.Y('TradeVolume', axis=alt.Axis(title='Shares Traded')),
    # color=alt.Color('rgb(74,82,90)'),
    tooltip=['Date', alt.Tooltip('Date', timeUnit="hours"), 'TradeVolume'],
).properties(
    title = "PredictIt Presidential Market Trading Volume"
)

alt.Chart(...)

The biggest event of election night was the calling of Florida for Trump, which occurred just before midnight Eastern Time. Traders promptly flooded the market and caused an outage at PredictIt. When the site came back online around 4AM there was significant pent up demand and volume surged. *Then* the presidential probabilities shook off their persistence: 

In [ ]:
#@title
#hide_input
alt.Chart(df).mark_line(size = 2).encode(
    x='Date',
    y=alt.Y('Win Probability',
        scale=alt.Scale(domain=[0,1])
    ),
    color='Candidate',
    tooltip=['Date', alt.Tooltip('Date', timeUnit="hours"), 'Win Probability', 'Candidate'],
).properties(
    title = "PredictIt Presidential Probabilites"
)

alt.Chart(...)

Trump's odds surged immediately from 44% to 64% and got as high as 72%. Then, as it became increasingly clear that Biden would win Michigan and Wisconsin Trump's odds plummeted into the 20% range. 

So throughout the campaign, the market stubbornly refused to budge in reaction to the passage of time, but when a surprising event occured the market *wildly* overreacted, causing a massive swing in prices that sustained for several hours before correcting. 

More wild than that though, is that as of this writing the market has a Biden win at 88%, almost 2 weeks after the race was called! 

PredictIt does have some [structural barriers](https://www.lesswrong.com/posts/c3iQryHA4tnAvPZEv/limits-of-current-us-prediction-markets-predictit-case-study) to translating prices precisely into probabilities, but these issues should show up in expanded bid/ask spreads. They shouldn't be enough to prevent the market from updating *at all* on the race being called for Biden by every major media source. 

Based on these patterns it seems reasonable to conclude:
1. PredictIt markets react strongly (sometimes too strongly) to unexpected events, but don't price in the reduction of uncertainty that occurs over time.
2. There's a certain amount of "nonreactive" money in the system, probably due to partisan bias, that can prevent a functionally certain event from reaching a reasonable price. This is probably exacerbated by PredictIt limiting the number of users who can participate on each contract and limiting how much they can bet, if enough partisans get in early, there's no way for smart people or smart money to come along later and correct their errors. 